In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は次のチャプターで説明を行います。
spark = SparkSession.builder \
    .appName("chapter2") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()

# 前回のチャプターいかがでしたか？
ちょっとdataframeの操作は慣れてなくて。。。
という人多かったんじゃないでしょうか

今回はデータをSQLで操作可能なSparkSQLを使いながら前回のチャプターと同じことをやっていこうと思います

# Sparkを用いたバッチにおけるデータエンジニアリング一連の流れ(復習)

1. データソースの読み込み(今回は、人口統計データ(/dataset/jinkou.csv))　ETL(Extract Transform Load)で言うEの部分
2. 変換を行う(集計等を行う)　DataFrame処理 or SQL処理の２パターンで実行可能 ETL(Extract Transform Load)で言うTの部分
3. カラムナーフォーマットへ変換する ETL(Extract Transform Load)で言うTの部分
4. 出力したデータをみんなに見やすくするため(BIツールから参照できるように)テーブルを作成する ETL(Extract Transform Load)で言うLの部分

よくある、関数の羅列をするのではなく、実業務に沿った形で流れを紹介していきます。

# データソースの読み込み

In [ ]:
from pyspark.sql.types import LongType, StructType, StructField, StringType
from pyspark.sql.functions import col

# カラム名、型、デフォルト値で設定していきます
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "SJIS").csv("/Users/yuki/pyspark_batch/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)

# 変換を行う(集計等を行う)

In [ ]:
# ここでDataFrameの処理と違いSQLの場合は、仮想的なテーブルjinkoを作成します
# テーブルを作成することでSQLを発行することができるようになります
df.createOrReplaceTempView("jinko")

In [ ]:
# 早速SQLを実行してみます
spark.sql("select * from jinko").show()

# 前のチャプターでDataframeベースで実行していた処理をSparkSQLを使って実装し直してみます。

```
df_after_t=df.where(df.gengo == "平成").groupBy("kenmei").agg(sf.avg("jinko_male").alias("male_avg"),sf.avg("jinko_female").alias("female_avg")).filter(df.kenmei != "人口集中地区以外の地区").sort("male_avg")

```

In [ ]:
# SQLで書き直すと、等価な処理は以下になります。

df_after_t=spark.sql(""" 

select kenmei,avg(jinko_male) as male_avg,avg(jinko_female) as female_avg 
 from jinko
  where gengo='平成' and kenmei != '人口集中地区以外の地区'
 group by kenmei
 order by male_avg

""")

df_after_t.show()

#後のカラムナーフォーマットへの出力からは前のチャプターと一緒です。

# SparkSQLを使うメリット

- SQLを使った処理はSQLファイルを外部に配置してそのファイルを読み込み実行することで汎用化がしやすい
- SQLになれた人であれば操作がしやすい
- データエンジニアリングとしては、SparkSQLを使ってシステム構築をするほうが汎用的で容易

# DataFrameを使うメリット
- データフレームにしかできないような仕事もある。例えば行列の変換などはdataframeのほうが実行しやすいのでデータサイエンスを好む人はDataFrameを使うほうが良い場合もある。

## 速度は変わる？
速度はどちらでも変わりません。内部的にはSQLはDataFrameとして処理され実行されます。